In [24]:
import pandas as pd
import glob
import os
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, classification_report

In [31]:
generos = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'ScienceFiction', 'TVMovie', 'Thriller', 'War', 'Western']

def read_parquet(pasta):
    arquivos_parquet = glob.glob(os.path.join(pasta, '*.parquet'))
    lista_df = [pd.read_parquet(arquivo) for arquivo in arquivos_parquet]
    df = pd.concat(lista_df, ignore_index=True)
    return df

In [46]:
df_parquet = read_parquet('./preprocess/movies')[generos]
label = 'Action'
features = generos.copy()
features.remove(label)

In [47]:
print("Antes do Undersampling: {}".format(len(df_parquet)))

rus = RandomUnderSampler(random_state=42)
X, y = rus.fit_resample(df_parquet[features], df_parquet[label])

print("Depois do Undersampling: {}".format(len(X_train)))

Antes do Undersampling: 38538
Depois do Undersampling: 8391


In [48]:
from sklearn.feature_selection import SelectKBest, chi2

# Seleção das melhores características
selector = SelectKBest(chi2, k=5)
selector.fit(X, y)

generos_selecionados = selector.get_feature_names_out()
print("Os 5 gêneros mais relevantes são:", generos_selecionados)

Os 5 gêneros mais relevantes são: ['Adventure' 'Documentary' 'Romance' 'ScienceFiction' 'Thriller']


In [39]:
# Definir e treinar o modelo Random Forest
model = RandomForestClassifier(criterion='entropy',random_state=42)
model.fit(X, y)

# Obter a importância das features
importances = model.feature_importances_

# Criar um dataframe para visualizar as importâncias
importancia_df = pd.DataFrame({'Gênero': features, 'Importância': importances})

# Ordenar as importâncias do maior para o menor
importancia_ordenada = importancia_df.sort_values(by='Importância', ascending=False)
print(importancia_ordenada)

# Definir um threshold (por exemplo, selecionar importância acima de 0.05)
threshold = 0.06
generos_selecionados = importancia_ordenada[importancia_ordenada['Importância'] > threshold]

# Gêneros selecionados
print("Gêneros selecionados com base no threshold:")
print(generos_selecionados['Gênero'])
X = X[generos_selecionados['Gênero']]


            Gênero  Importância
0        Adventure     0.179746
16        Thriller     0.100933
4      Documentary     0.099970
10          Horror     0.071353
5            Drama     0.068280
2           Comedy     0.067680
3            Crime     0.061278
14  ScienceFiction     0.052271
13         Romance     0.050315
12         Mystery     0.041129
18         Western     0.036408
6           Family     0.031501
1        Animation     0.024399
7          Fantasy     0.023804
11           Music     0.021680
8          Foreign     0.021641
17             War     0.019565
9          History     0.014781
15         TVMovie     0.013268
Gêneros selecionados com base no threshold:
0       Adventure
16       Thriller
4     Documentary
10         Horror
5           Drama
2          Comedy
3           Crime
Name: Gênero, dtype: object


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [50]:
model = CategoricalNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))



Accuracy: 0.7428412566027245
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.76      0.75      1799
           1       0.75      0.73      0.74      1798

    accuracy                           0.74      3597
   macro avg       0.74      0.74      0.74      3597
weighted avg       0.74      0.74      0.74      3597

